# Chapter 10: Architecture and Design Patterns

## 🏗️ Learning Objectives
- Understand core secure design principles
- Apply security architecture patterns
- Analyze architectural security risks
- Practice secure design evaluation

In [1]:
# Setup for Chapter 10 architecture examples
import os
import sys
import platform
from datetime import datetime

print("Chapter 10: Architecture and Design Patterns")
print("=" * 45)
print(f"Platform: {platform.system()}")
print(f"Python Version: {sys.version.split()[0]}")
print("Ready for secure architecture analysis!")

Chapter 10: Architecture and Design Patterns
Platform: Windows
Python Version: 3.13.5
Ready for secure architecture analysis!


## 1. Core Security Design Principles

The CourseGuide outlines key principles for secure system design:
- **Least Privilege**: Minimal permissions needed
- **Defense-in-Depth**: Layered security controls
- **Fail Safe**: Secure failure modes
- **Complete Mediation**: Check every access

In [2]:
# Demonstrate Least Privilege Principle
print("1. LEAST PRIVILEGE PRINCIPLE")
print("-" * 30)

class FileReader:
    """Limited file reading class - only read permission"""
    def __init__(self, allowed_extensions=['.txt', '.log']):
        self.allowed_extensions = allowed_extensions
    
    def read_file(self, filename):
        # Check file extension (privilege limitation)
        ext = os.path.splitext(filename)[1].lower()
        if ext not in self.allowed_extensions:
            raise PermissionError(f"Not authorized to read {ext} files")
        
        print(f"✅ Reading {filename} (authorized extension)")
        return f"Content of {filename}"

# Demo least privilege
reader = FileReader()
try:
    reader.read_file("data.txt")  # Allowed
    reader.read_file("config.exe")  # Blocked
except PermissionError as e:
    print(f"❌ Access denied: {e}")

print("\n💡 Least privilege limits potential damage from compromise")

1. LEAST PRIVILEGE PRINCIPLE
------------------------------
✅ Reading data.txt (authorized extension)
❌ Access denied: Not authorized to read .exe files

💡 Least privilege limits potential damage from compromise


## 2. Exercise 1: Language Migration Risks

**CourseGuide Exercise**: *What security risks are involved when converting a codebase to a different language?*

In [ ]:
# Exercise 1: Language Migration Security Analysis
print("2. LANGUAGE MIGRATION SECURITY RISKS")
print("-" * 40)

migration_risks = {
    "Memory Management": [
        "C/C++ to Python: Manual memory → garbage collection",
        "Risk: Loss of precise memory control",
        "Benefit: Elimination of buffer overflows"
    ],
    "Type Systems": [
        "JavaScript to TypeScript: Dynamic → static typing",
        "Risk: Runtime type errors become compile-time",
        "Benefit: Early detection of type mismatches"
    ],
    "Library Ecosystems": [
        "Java to Python: Different library security models",
        "Risk: New libraries may have unknown vulnerabilities",
        "Mitigation: Thorough security review of dependencies"
    ],
    "Runtime Environments": [
        "Native to VM: Different attack surfaces",
        "Risk: New runtime-specific vulnerabilities",
        "Example: JVM sandbox escapes vs native code injection"
    ]
}

for category, details in migration_risks.items():
    print(f"\n📋 {category}:")
    for detail in details:
        print(f"   • {detail}")

print("\n💡 Key Takeaway: Language migration requires security re-evaluation")

## 3. Exercise 2: Platform Porting Risks

**CourseGuide Exercise**: *What security risks are involved in porting Unix applications to Windows and vice versa?*

In [ ]:
# Exercise 2: Platform Porting Security Analysis
print("3. PLATFORM PORTING SECURITY RISKS")
print("-" * 40)

# Unix to Windows porting risks
unix_to_windows = {
    "File Permissions": "Unix rwx → Windows ACLs (more complex)",
    "Path Separators": "/ vs \\ - path traversal vulnerabilities",
    "Case Sensitivity": "Unix case-sensitive → Windows case-insensitive",
    "Process Model": "fork/exec → CreateProcess (different security contexts)",
    "Registry vs Files": "Config files → Windows Registry (new attack surface)"
}

# Windows to Unix porting risks  
windows_to_unix = {
    "Privilege Model": "Windows tokens → Unix uid/gid (simpler but different)",
    "File Extensions": "Windows relies on .exe → Unix executable bit",
    "Drive Letters": "C:\\ → /mount/point (path confusion)",
    "Network Security": "Windows domain auth → Unix varied auth systems",
    "Shared Libraries": "DLLs → .so files (different loading mechanisms)"
}

print("🔄 Unix → Windows Risks:")
for risk, description in unix_to_windows.items():
    print(f"   • {risk}: {description}")

print("\n🔄 Windows → Unix Risks:")
for risk, description in windows_to_unix.items():
    print(f"   • {risk}: {description}")

print("\n💡 Platform differences create new attack vectors")

## 4. Distrustful Decomposition Pattern

The CourseGuide describes **distrustful decomposition** - separating functionality into mutually distrustful modules.

In [ ]:
# Demonstrate Distrustful Decomposition
print("4. DISTRUSTFUL DECOMPOSITION PATTERN")
print("-" * 40)

class UserValidator:
    """Low-privilege user validation module"""
    def validate_user(self, username, password):
        # Never trust input - always validate
        if not username or len(username) < 3:
            return False, "Invalid username"
        if not password or len(password) < 8:
            return False, "Password too short"
        
        print(f"✅ User validation passed for: {username}")
        return True, "Valid credentials"

class PrivilegedAction:
    """High-privilege action module - distrusts validator"""
    def __init__(self):
        self.validator = UserValidator()
    
    def perform_admin_action(self, username, password, action):
        # Don't trust the validator - verify independently
        is_valid, message = self.validator.validate_user(username, password)
        
        # Additional privilege checks (distrust the validator)
        if not is_valid:
            print(f"❌ Action denied: {message}")
            return False
        
        # Re-validate at privilege boundary
        if username != "admin":  # Additional check
            print(f"❌ Insufficient privileges for {username}")
            return False
            
        print(f"🔐 Admin action '{action}' executed by {username}")
        return True

# Demo distrustful decomposition
admin_system = PrivilegedAction()

# Test cases
admin_system.perform_admin_action("user", "password123", "delete_database")
admin_system.perform_admin_action("admin", "strongpassword", "backup_database")

print("\n💡 Each module validates independently - no blind trust")

## 5. Clear Sensitive Information Pattern

The CourseGuide emphasizes clearing sensitive data when no longer needed.

In [ ]:
# Demonstrate Clear Sensitive Information
print("5. CLEAR SENSITIVE INFORMATION PATTERN")
print("-" * 40)

import os

class SecureCredentialHandler:
    """Handles credentials with proper cleanup"""
    def __init__(self):
        self.password = None
        self.session_token = None
    
    def authenticate(self, username, password):
        print(f"🔐 Authenticating user: {username}")
        
        # Store password temporarily for processing
        self.password = password
        
        # Simulate authentication process
        if len(password) >= 8:
            self.session_token = "session_" + os.urandom(8).hex()
            print(f"✅ Authentication successful")
            
            # IMPORTANT: Clear sensitive data immediately
            self._clear_sensitive_data()
            return self.session_token
        else:
            print("❌ Authentication failed")
            self._clear_sensitive_data()
            return None
    
    def _clear_sensitive_data(self):
        """Explicitly clear sensitive information"""
        if self.password:
            # Overwrite password memory (best effort in Python)
            self.password = "0" * len(self.password)
            self.password = None
            print("🧹 Password cleared from memory")
    
    def logout(self):
        """Clear session data on logout"""
        if self.session_token:
            print(f"🔓 Clearing session: {self.session_token[:12]}...")
            self.session_token = None
            print("✅ Session cleared")

# Demo secure credential handling
auth_handler = SecureCredentialHandler()

# Successful authentication
token = auth_handler.authenticate("user1", "strongpassword")
print(f"Received token: {token[:12] if token else 'None'}...")

# Logout and cleanup
auth_handler.logout()

print("\n💡 Always clear sensitive data when no longer needed")

## 6. Exercise 3: Clipper Chip Analysis

**CourseGuide Exercise**: *What architectural/design flaws exist in the 1990s Clipper encryption chip?*

In [ ]:
# Exercise 3: Clipper Chip Design Flaw Analysis
print("6. CLIPPER CHIP DESIGN FLAWS")
print("-" * 35)

clipper_flaws = {
    "1. Violation of Open Design Principle": [
        "Secret Skipjack cipher (not open to review)",
        "Violates Kerckhoff's Principle",
        "Security through obscurity approach"
    ],
    "2. Key Escrow Architecture": [
        "Government holds decryption keys",
        "Single point of failure/compromise",
        "Trust boundary includes potential adversary"
    ],
    "3. Psychological Acceptability Issues": [
        "Users know government can decrypt",
        "Reduces user confidence and adoption",
        "Creates incentive to find workarounds"
    ],
    "4. Manufacturing Trust Issues": [
        "Keys determined at manufacture time",
        "Supply chain trust requirements",
        "No user control over key generation"
    ]
}

for flaw_category, details in clipper_flaws.items():
    print(f"\n❌ {flaw_category}:")
    for detail in details:
        print(f"   • {detail}")

print("\n🎯 Core Problem: Conflicting trust requirements")
print("   Users need privacy vs Government needs access")
print("\n💡 Modern lesson: Backdoors weaken security for everyone")

## 7. Summary: Secure Architecture Principles

Key takeaways from Chapter 10:
- Architecture flaws are harder to fix than code bugs
- Security principles guide design decisions
- Trust boundaries must be carefully designed
- Sensitive data requires explicit cleanup

In [ ]:
# Chapter 10 Summary
print("7. CHAPTER 10 SUMMARY")
print("-" * 25)

principles_demonstrated = {
    "✅ Least Privilege": "Limited file reader permissions",
    "✅ Distrustful Decomposition": "Validator + privilege checker separation", 
    "✅ Clear Sensitive Information": "Explicit password/session cleanup",
    "✅ Open Design Analysis": "Clipper chip flaw identification",
    "✅ Migration Risk Assessment": "Language and platform security impacts"
}

print("\n🏗️ Secure Architecture Principles Applied:")
for principle, example in principles_demonstrated.items():
    print(f"   {principle}: {example}")

print("\n🎯 Key Architecture Security Insight:")
print("   'A security misstep in design is much harder to patch'")
print("   - Plan security from the beginning")
print("   - Use established secure design patterns")
print("   - Always consider trust boundaries")

print("\n✅ Chapter 10 Complete - Ready for Chapter 11!")